In [1]:
import torch
import torch.nn as nn
from torchtext import data
from torchtext.data import Field, RawField
import numpy as np
import json

from signjoey.data import load_data
from signjoey.model import build_model
from signjoey.helpers import load_config

2022-12-30 14:07:38.680059: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-30 14:07:38.680091: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-30 14:07:45.779305: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-12-30 14:07:45.830079: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-12-30 14:07:46.429806: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-12-30 14:07:46.429826: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pol): /proc/driver/nvidia/ve

In [2]:
cfg_file = r"./configs/prob2text/p1/tlp_prob.yaml"
cfg = load_config(cfg_file)

In [3]:
p_train_data, dev_data, test_data, gls_vocab, txt_vocab = load_data(
    data_cfg=cfg["data"],
    data_type="prob"
)

Using mBART vocabulary? False
loading ./data/probs/tlp_output_train.json
loading ./data/probs/tlp_output_dev.json
loading ./data/probs/tlp_output_test.json


In [19]:
i = iter(p_train_data.sgn)

In [25]:
next(i).shape

torch.Size([119, 1116])

In [16]:
p_train_data.fields

{'sequence': <torchtext.data.field.RawField at 0x7fa0c44a8580>,
 'signer': <torchtext.data.field.RawField at 0x7fa0c44a8790>,
 'sgn': <torchtext.data.field.Field at 0x7fa0c5d18f70>,
 'gls': <torchtext.data.field.Field at 0x7fa0c5d18af0>,
 'txt': <torchtext.data.field.Field at 0x7fa0c5d18fd0>}

In [5]:
next(train_data.sgn).shape

torch.Size([86, 1024])

In [5]:
model = build_model(
    cfg=cfg["model"],
    gls_vocab=gls_vocab,
    txt_vocab=txt_vocab,
    sgn_dim=sum(cfg["data"]["feature_size"])
    if isinstance(cfg["data"]["feature_size"], list)
    else cfg["data"]["feature_size"],
    do_recognition=False,
    do_translation=True,
)

Freezing transformer: False
 --------> Yeah, using super prob model!!!
Special tokens:
BOS = 2
PAD = 1
EOS = 3


[('encoder.layers.0.layer_norm.weight',
  Parameter containing:
  tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 